In [1]:
import pandas as pd

In [2]:
from nsepy import get_history as gh
import datetime as dt

In [3]:

stk_data = pd.read_csv("Tatacoffee13_21.csv")
stk_data['Date'] = pd.to_datetime(stk_data['Date'])

# Filter data for the specified date range
start_date = dt.datetime(2013, 6, 1)
end_date = dt.datetime(2022, 2, 11)

date_range = stk_data[(stk_data['Date'] >= start_date) &(stk_data['Date'] <= end_date)]


In [4]:
date_range

,Date,Open,High,Low,Close
105,2013-06-03,1485.00,1497.50,1451.00,1467.40
106,2013-06-04,1453.00,1509.00,1453.00,1462.15
107,2013-06-05,1449.00,1470.00,1449.00,1459.50
108,2013-06-06,1465.00,1470.00,1455.35,1463.70
109,2013-06-07,1466.20,1484.85,1446.00,1451.70
...,...,...,...,...,...
2220,2021-12-22,202.90,207.80,201.35,205.00
2221,2021-12-23,206.00,206.85,202.05,202.95
2222,2021-12-24,203.90,203.90,199.35,201.00
2223,2021-12-27,200.00,222.00,196.00,218.35


In [5]:
stk_data=stk_data[["Open","High","Low","Close"]]
stk_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    2225 non-null   float64
 1   High    2225 non-null   float64
 2   Low     2225 non-null   float64
 3   Close   2225 non-null   float64
dtypes: float64(4)
memory usage: 69.7 KB


In [6]:
column="Close"

In [7]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (2225, 4)


In [8]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])
data1

,Open,High,Low,Close
0,0.849900,0.845408,0.856100,0.854203
1,0.856394,0.967399,0.861040,0.974481
2,0.988480,0.996439,0.984959,0.986240
3,0.985483,0.968105,0.960760,0.956749
4,0.955669,0.975319,0.955033,0.967132
...,...,...,...,...
2220,0.095842,0.096329,0.096510,0.097323
2221,0.097777,0.095745,0.096951,0.096041
2222,0.096466,0.093934,0.095252,0.094821
2223,0.094031,0.105047,0.093143,0.105673


In [9]:
len(data1)

2225

In [10]:
training_size = round(len(data1 ) * 0.80)
print(training_size)
X_train=data1[:training_size]
X_test=data1[training_size:]
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
y_train=data1[:training_size]
y_test=data1[training_size:]
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

1780
X_train length: (1780, 4)
X_test length: (445, 4)
y_train length: (1780, 4)
y_test length: (445, 4)


In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}

In [13]:
def cominbation(dataset, listt, exog_cols):
    from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
    import numpy as np
    from statsmodels.tsa.statespace.varmax import VARMAX
    import pandas as pd
    
    datasetTwo = dataset[listt] # Target variables
    exog_data = dataset[exog_cols] # Exogenous variables
    test_obs = 28
    
    #Train/Test Split for Targets
    train = datasetTwo[:-test_obs]
    test = datasetTwo[-test_obs:]
    
    # Train/Test Split for Exogenous
    exog_train = exog_data[:-test_obs]
    exog_test = exog_data[-test_obs:]
    
    aic = np.inf
    order = None
    model_result = None
    
    # Grid Search with 9 combinations
    for p in range(1, 4):  # p: 1, 2, 3
        for q in range(0, 3):  # q: 0, 1, 2
            try:
                model = VARMAX(train, exog=exog_train, order=(p, q), trend='c')
                result = model.fit(disp=False) 
                
                print(f"order (p={p}, q={q}) completed. AIC: {result.aic:.2f}")
                
                if result.aic < aic:
                    aic = result.aic
                    order = (p, q)
                    model_result = result
            except Exception as e:
                print(f"order (p={p}, q={q}) failed: {e}")
                continue

    # After all 9 order,only the best result
    if model_result is not None:
        print(f"\n--- Grid Search Completed ---")
        print(f"Best Model (Order): {order}")
        
        forecast = model_result.get_forecast(steps=test_obs, exog=exog_test).predicted_mean
        rmse = round(mean_squared_error(test, forecast, squared=False), 2)
        mape = mean_absolute_percentage_error(test, forecast)
        
        # Save results into the dictionary only once
        performance["Model"].append("VARMAX_Exog")
        performance["RMSE"].append(rmse)
        performance["MaPe"].append(mape)
        performance["Lag"].append(str(order))
        performance["Test"].append(test_obs)
        
        perf = pd.DataFrame(performance)
        return perf, model_result, forecast
    

In [14]:
listt = ["Close", "High"]
my_exog_cols = ["Open", "Low"]


In [15]:
perf, result, forecast = cominbation(data1, listt, my_exog_cols)

order (p=1, q=0) completed. AIC: -36576.07


C:\Users\user\Anaconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:163: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  EstimationWarning)


order (p=1, q=1) completed. AIC: -36567.67


C:\Users\user\Anaconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:163: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  EstimationWarning)
C:\Users\user\Anaconda3\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


order (p=1, q=2) completed. AIC: -36563.29


...

order (p=2, q=2) completed. AIC: -36463.39
order (p=3, q=0) completed. AIC: -36072.72


C:\Users\user\Anaconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:163: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  EstimationWarning)


order (p=3, q=1) completed. AIC: -36081.62


C:\Users\user\Anaconda3\lib\site-packages\statsmodels\tsa\statespace\varmax.py:163: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  EstimationWarning)


order (p=3, q=2) completed. AIC: -36103.74

--- Grid Search Completed ---
Best Model (Order): (1, 0)


In [16]:
data1

,Open,High,Low,Close
0,0.849900,0.845408,0.856100,0.854203
1,0.856394,0.967399,0.861040,0.974481
2,0.988480,0.996439,0.984959,0.986240
3,0.985483,0.968105,0.960760,0.956749
4,0.955669,0.975319,0.955033,0.967132
...,...,...,...,...
2220,0.095842,0.096329,0.096510,0.097323
2221,0.097777,0.095745,0.096951,0.096041
2222,0.096466,0.093934,0.095252,0.094821
2223,0.094031,0.105047,0.093143,0.105673


In [17]:
perf

,Model,RMSE,MaPe,Lag,Test
0,VARMAX_Exog,0.0,0.014841,"(1, 0)",28
